# Read a XML file
cool point: light file, contain the taxonomy, DNA sequence and a lot of other info

In [1]:
import xml.etree.ElementTree as ET
tree = ET.parse('sequence.gbc.xml')
root = tree.getroot()

In [2]:
tax = root.find("./INSDSeq/INSDSeq_taxonomy").text
print(">", tax)

seq = root.find("./INSDSeq/INSDSeq_sequence").text
print(seq)

> Bacteria; Bacillota; Bacilli; Bacillales; Bacillaceae; Anaerobacillus
atgcgtcaaatagctttctacggtaagggtggtattggtaagtctacgacttctcaaaatacgttagttcaattagcaaaaaattatggtcaagaaatcatgatcgttggttgtgacccgaaggctgactctacgagactagtattaaatacaaaaatgcaacaaactgttttatctttagctgctgaaaaaggaacagttgaagatttagaactacatgaagtagcgcaaaatggttttgctggcattgaatgtgttgagtctggtgggccagaaccaggtgtagggtgtgcaggtcgaggaattatcacatctattaacttccttgaagaagaaggcgcttatgaaggaaaagactttatttcttatgacgtattaggggacgttgtttgtggtggatttgcgatgccaatccgtgaaaataaagcccaagaaatttacatcgtatgttctggtgaaatgatggcaatgtacgcagctaacaacattgcaaaaggggtattaaaatatgcacacagtggtggcgttcgtttaggtggtttaatttgtaacagccgtaacacagaccgagaagcagaccttattacagagttagcaagacgattaaatactcaaatgatccattttgttcctcgtgacaatattgtacagcatgctgagttaagaagaatgacagttgctcaatatgctcctgaaagtcaacaagcaaaagagtacgatatgttagctaagaaaattttagataatgatatgatgacaattcctacgccaatcgaaactgatgagttagaaggactattaatggaattcggtattattgaggatgaagaaactcaacttaaaaaattagcagcaaaagaagcagctagtcagtaa


# Search in IPG

In [3]:
# Import packages
from Bio import Entrez
from Bio import SeqIO

Entrez.email = "remi.legrand38@gmail.com.com"

In [4]:
# function to search data on NCBI
def search_id(query,number_seq):
    handle = Entrez.esearch(db="ipg", term=query, retmax=number_seq) # number of sequences
    record = Entrez.read(handle)
    handle.close()
    return record["IdList"]

In [5]:
# create a list containing the ID
query = "((nifH[Gene Name]) AND 100:350[Sequence Length]) NOT uncultured"
seq_id = search_id(query,20000)
print(seq_id)
len(seq_id)

['127971653', '127920796', '127919708', '127531281', '127125494', '127122484', '124605006', '123699430', '123240796', '123238747', '122968989', '15644230', '651109310', '651099322', '539807534', '427539919', '427349194', '427347870', '427347502', '427307658', '123485017', '120090222', '108730382', '100437320', '77311835', '77141912', '75522125', '75003110', '68375739', '56757924', '56229910', '55856836', '55125244', '52011023', '46203031', '44492778', '41802330', '41802272', '41801635', '40163719', '32423034', '32005358', '32004869', '31938024', '20398129', '18484032', '15422654', '444818', '296546', '173295', '107895', '94880', '568627389', '427352784', '427352231', '427323180', '427300029', '427299397', '349975272', '328571347', '319883179', '319881941', '240261437', '238099139', '227006154', '214571543', '214571338', '200599404', '199647670', '195996521', '195508867', '195498420', '194947467', '178280531', '159366625', '152984701', '152983676', '152982965', '152982650', '152982176',

3944

At least I obtain the right ID for the IPG database

In [ ]:
# create a list containing the ID
query = "((nifH[Gene Name]) AND 100:350[Sequence Length]) NOT uncultured"
seq_id = search_id(query,20000)
print(seq_id)
len(seq_id)